<a href="https://colab.research.google.com/github/mtszkw/kaggle-lish-moa/blob/main/kaggle_lish_moa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
input_dir = '/content/'
# input_dir = '../input/lish-moa'

In [2]:
# https://www.kaggle.com/yasufuminakama/moa-pytorch-nn-starter

import os
import sys

!pip -q install --upgrade kaggle

%env KAGGLE_USERNAME=mtszkw
%env KAGGLE_KEY=4542560feb261d28dd30710ffc95cb41

# Dataset
# https://www.kaggle.com/c/lish-moa
!kaggle competitions download -c lish-moa

# iterative-stratification
# https://www.kaggle.com/yasufuminakama/iterative-stratification
!kaggle datasets download -d yasufuminakama/iterative-stratification

!unzip -q '/content/*.zip'

for x in os.listdir('/content'):
    if x.endswith(".zip"):
        os.remove(os.path.join('/content', x))

os.listdir('/content')

env: KAGGLE_USERNAME=mtszkw
env: KAGGLE_KEY=4542560feb261d28dd30710ffc95cb41
  0% 0.00/38.9k [00:00<?, ?B/s]
100% 38.9k/38.9k [00:00<00:00, 35.4MB/s]
  0% 0.00/257k [00:00<?, ?B/s]
100% 257k/257k [00:00<00:00, 83.9MB/s]
  0% 0.00/535k [00:00<?, ?B/s]
100% 535k/535k [00:00<00:00, 75.3MB/s]
  0% 0.00/9.16M [00:00<?, ?B/s]
100% 9.16M/9.16M [00:00<00:00, 151MB/s]
  0% 0.00/233k [00:00<?, ?B/s]
100% 233k/233k [00:00<00:00, 73.4MB/s]
 78% 43.0M/54.8M [00:00<00:00, 179MB/s]
100% 54.8M/54.8M [00:00<00:00, 214MB/s]
  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 16.2MB/s]

6 archives were successfully processed.


['.config',
 'train_features.csv',
 'train_targets_scored.csv',
 'iterative-stratification-master',
 'sample_submission.csv',
 'train_targets_nonscored.csv',
 'test_features.csv',
 'train_drug.csv',
 'sample_data']

In [3]:
import numpy as np
import pandas as pd
from typing import Optional

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
import torch.nn.functional as F

!pip -q install pytorch-lightning
import pytorch_lightning as pl

!pip -q install '/content/iterative-stratification-master'
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

     |████████████████████████████████| 552kB 13.7MB/s 
     |████████████████████████████████| 92kB 12.3MB/s 
     |████████████████████████████████| 829kB 59.5MB/s 
     |████████████████████████████████| 276kB 48.0MB/s 


In [61]:
train_features = pd.read_csv(os.path.join(input_dir, 'train_features.csv'))
train_targets_scored = pd.read_csv(os.path.join(input_dir, 'train_targets_scored.csv'))

test_features = pd.read_csv(os.path.join(input_dir, 'test_features.csv'))

# train_targets_nonscored = pd.read_csv(os.path.join(input_dir, 'train_targets_nonscored.csv'))
# submission = pd.read_csv(os.path.join(input_dir, 'sample_submission.csv'))

print(f'Train: {train_features.shape}, test: {test_features.shape}')

Train: (23814, 876), test: (3982, 876)


In [62]:
train_features.dtypes

sig_id      object
cp_type     object
cp_time      int64
cp_dose     object
g-0        float64
            ...   
c-95       float64
c-96       float64
c-97       float64
c-98       float64
c-99       float64
Length: 876, dtype: object

### Dataset

In [63]:
# class MoATrainDataset(torch.utils.data.Dataset):
#     def __init__(self, train_features: pd.DataFrame, train_targets: pd.DataFrame):
#         self.features = train_features
#         self.targets = train_targets

#         self.transform = transforms.Compose([transforms.ToTensor()])

#     def __len__(self):
#         return len(self.features)

#     def __getitem__(self, idx):
#         x = self.transform(self.features.iloc[idx])
#         y = self.transform(self.targets.iloc[idx])
#         return x, y

In [64]:
# class MoATestDataset(torch.utils.data.Dataset):
#     def __init__(self, data: pd.DataFrame, target_cols: List):
#         self.target_cols = target_cols
        
#         self.features = test_features

#     def __len__(self):
#         return len(self.features)

#     def __getitem__(self, idx):
#         X = self.features[idx]
#         return X

In [65]:
# class MoADataModule(pl.LightningDataModule):
#     def __init__(self, batch_size, train_ds, valid_ds, test_ds):
#         super().__init__()
#         self.batch_size = batch_size
#         self.train_ds = train_ds
#         self.valid_ds = valid_ds
#         self.test_ds = test_ds

#     def setup(self, stage: Optional[str] = None):
#         if stage == 'fit' or stage is None:
#             pass
#         if stage == 'test' or stage is None:
#             pass

#     def train_dataloader(self):
#         return torch.utils.data.DataLoader(
#             self.train_ds, shuffle=False, batch_size=self.batch_size)

#     def valid_dataloader(self):
#         return torch.utils.data.DataLoader(
#             self.valid_ds, shuffle=False,  batch_size=self.batch_size)

#     def test_dataloader(self):
#         return torch.utils.data.DataLoader(
#             self.test_ds, shuffle=False, batch_size=self.batch_size)
        

### Model

In [66]:
class MoANeuralNetwork(pl.LightningModule):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.net = self.get_model(input_size, output_size)

    def get_model(self, input_size, output_size):
        dropout = .5
        hidden_size = 512

        return nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(dropout),
            nn.PReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(dropout),
            nn.PReLU(),
            nn.Linear(hidden_size, output_size))

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.net.parameters(), lr=0.01, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
        return [optimizer], [scheduler]

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = torch.nn.BCEWithLogitsLoss()(y_hat, y)

        self.log_dict({'loss': loss})
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = torch.nn.BCEWithLogitsLoss()(y_hat, y)

        self.log_dict({'val_loss': loss})
        return loss


### Training

In [67]:
# ref: https://www.kaggle.com/c/lish-moa/discussion/180165
# check if labels for 'ctl_vehicle' are all 0.
train_data = train_features.merge(train_targets_scored, on='sig_id')
train_data = train_data[train_data['cp_type']!='ctl_vehicle'].reset_index(drop=True)

test_data  = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

print(f'Train data: {train_data.shape}, test data: {test_data.shape}')

Train data: (21948, 1082), test data: (3624, 876)


In [68]:
target_cols = [c for c in train_targets_scored.columns if c not in ['sig_id']]

len(target_cols)

206

In [69]:
def cate2num(df):
    df['cp_time'] = df['cp_time'].map({24: 0, 48: 1, 72: 2})
    df['cp_dose'] = df['cp_dose'].map({'D1': 3, 'D2': 4})
    return df

train_data = cate2num(train_data)
test_data  = cate2num(test_data)
feature_cols = [c for c in train_data.columns if train_data.dtypes[c] != 'object']
feature_cols = [c for c in feature_cols if c not in target_cols]

len(feature_cols)

874

In [ ]:
folds = train_data.copy()

k_fold = MultilabelStratifiedKFold(n_splits=5, shuffle=False, random_state=42)
for n, (train_index, valid_index) in enumerate(k_fold.split(folds, folds[target_cols])):

    df_train = folds.iloc[train_index]
    df_valid = folds.iloc[valid_index]

    train_y = torch.tensor(df_train[target_cols].values.astype(np.float32))
    train_x = torch.tensor(df_train[feature_cols].values.astype(np.float32)) 
    train_tensor = torch.utils.data.TensorDataset(train_x, train_y) 

    valid_y = torch.tensor(df_valid[target_cols].values.astype(np.float32))
    valid_x = torch.tensor(df_valid[feature_cols].values.astype(np.float32)) 
    valid_tensor = torch.utils.data.TensorDataset(valid_x, valid_y) 

    # train_x = df_train.drop(target_cols, axis=1)
    # train_y = df_train[target_cols]

    # valid_x = df_valid.drop(target_cols, axis=1)
    # valid_y = df_valid[target_cols]

    train_dl = torch.utils.data.DataLoader(train_tensor, shuffle=False, batch_size=16)    
    valid_dl = torch.utils.data.DataLoader(valid_tensor, shuffle=False, batch_size=16)

    print('Train', train_x.shape, train_y.shape)
    print('Valid', valid_x.shape, valid_y.shape)

    trainer = pl.Trainer(gpus=1, max_epochs=5)
    lit_module = MoANeuralNetwork(input_size=train_x.shape[1], output_size=train_y.shape[1])
    trainer.fit(lit_module, train_dl, valid_dl)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 818 K 


Train torch.Size([17558, 874]) torch.Size([17558, 206])
Valid torch.Size([4390, 874]) torch.Size([4390, 206])


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Train torch.Size([17559, 874]) torch.Size([17559, 206])
Valid torch.Size([4389, 874]) torch.Size([4389, 206])



  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 818 K 


In [ ]:
# trainer = pl.Trainer()

# dataset = 
# model = MoANeuralNetwork()

# trainer.fit()